# Deep Learning Optimization

#### Goal of This File:

##### 1. Import Libraries and Helper Functions ==> Data Sourcing and Munging ==> Utility Functions ==> Data Preparation and Pre-Processing

##### 2. Confirm Findings From ML File ==> Sklearn Embeddings vs Gensim Embeddings ==> Sentiment Polarity Final Score ==> 3 labels vs 5 labels

##### 3. XGBoost

##### 4. Simple RNN

##### 5. LSTM ==> Bidirectional LSTM

##### 6. Sequence-to-Vector Transformers

##### 7. Conclusion

## 1. Import Libraries and Helper Functions

In [ ]:
import pandas as pd
import requests
import os
from nlp_ai_utils import *
from chunks_urls import CHUNKS_URLS
URLS = CHUNKS_URLS
LIMIT = 60

### 1.1 Data Sourcing and Munging

#### 1.1.1 Utility Functions

In [ ]:
def get_chunks(urls):
  #downloads all data from their urls
  for i,url in enumerate(urls):
    file_name = "chunk_" + str(i + 1)
    #checks if file already exists
    if os.path.exists(file_name):
      continue
    r = requests.get(url)
    print(f"Downloading {file_name}...")
    with open(file_name, 'wb') as fd:
      for chunk in r.iter_content():
        #save file in the current directory of the notebook
        fd.write(chunk)

In [ ]:
def read_chunks(files):
    #reads chunks csvs and converts them to a dataframe format
    final_df = []
    for file in files:
        df = pd.read_csv(file)
        final_df.append(df)
    #function returns a list of dfs, or one df if only one input is given
    return final_df

In [ ]:
def group_up_chunks(dfs):
    #adds up all dataframes together
    return pd.concat(dfs)

### 1.2 Data Preparation and Pre-processing